In [1]:
import setup

setup.init_django()

In [2]:
from rag import (
    db as rag_db, 
    engines as rag_engines,
    settings as rag_settings, 
    updaters as rag_updaters,
    patches as rag_patches,
)

In [3]:
from typing import Optional, Union
from sqlalchemy import create_engine, text

In [4]:
import logging
logging.basicConfig(level=logging.INFO)

rag_settings.init()
rag_db.init_vector_db()
rag_updaters.update_llama_index_documents(use_saved_embeddings=True)

Vector database 'coffeshop_vector_db' initialized successfully.
Syncing 1 docs for EmployeeRole


INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"


Syncing 1 docs for Employee
Syncing 1 docs for ProductType
Syncing 1 docs for Product
Syncing 1 docs for InventoryItem
Syncing 1 docs for ProductInventoryRequirement


INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"


Sync complete for all specified models.


In [5]:
vector_index = rag_engines.get_semantic_query_index()
semantic_query_retriever = rag_engines.get_semantic_query_retriever_engine()
sql_query_engine = rag_engines.get_sql_query_engine()

In [9]:
print(rag_settings.VECTOR_DB_NAME, rag_settings.VECTOR_TABLE_NAMES)

coffeshop_vector_db {'EmployeeRole': 'employee_role_vectors', 'Employee': 'employee_vectors', 'ProductType': 'product_type_vectors', 'Product': 'product_vectors', 'InventoryItem': 'inventory_item_vectors', 'ProductInventoryRequirement': 'product_inventory_req_vectors'}


In [10]:
from llama_index.core.tools import QueryEngineTool

vector_tool = QueryEngineTool.from_defaults(
    query_engine=semantic_query_retriever,
    description=(
        f"Useful for answering semantic questions about different blog posts"
    ),
)

In [11]:
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over "
        "tables containing:EmployeeRole, Employee, ProductType, Product, InventoryItem, ProductInventoryRequirement."
    ),
)

In [12]:
query_engine = rag_patches.MySQLAutoVectorQueryEngine(
    sql_tool, 
    vector_tool,
)

In [13]:
response = query_engine.query(
    "what is americano?"
)

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.sql_join_query_engine:> Querying other query engine: The question 'what is americano?' is semantic in nature and likely refers to a type of coffee, which could be the subject of a blog post.


Querying other query engine: The question 'what is americano?' is semantic in nature and likely refers to a type of coffee, which could be the subject of a blog post.


INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


In [14]:
response.response

'An americano is something that requires 50.00 grams of Coffe 60/40.'

In [17]:
response = query_engine.query(
    "Who is the bartender?"
)

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.sql_join_query_engine:> Querying SQL database: The question 'Who is the bartender?' implies a query about a specific role, which could be related to an 'EmployeeRole' in a database context.
INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'coffeshop_employeerole' has columns: id (BIGINT), name (VARCHAR(100)), description (TEXT), embedding (VECTOR(1024)), .

Table 'coffeshop_employee' has columns: id (BIGINT), name (VARCHAR(200)), hire_date (DATE), hourly_rate (NUMERIC(6, 2)), is_active (BOOLEAN), embedding (VECTOR(1024)), role_id (BIGINT),  and foreign keys: ['role_id'] -> coffeshop_employeerole.['id'].

Table 'coffeshop_producttype' has columns: id (BIGINT), name (VARCHAR(100)), description (TEXT), embedding (VECTOR(1024)), .

Table 'coffeshop_product' has columns: id (BIGINT), name (VARCHAR(200)), price (NUMERIC(6, 2)), descript

Querying SQL database: The question 'Who is the bartender?' implies a query about a specific role, which could be related to an 'EmployeeRole' in a database context.


INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


SQL query: SELECT coffeshop_employee.name
FROM coffeshop_employee
JOIN coffeshop_employeerole ON coffeshop_employee.role_id = coffeshop_employeerole.id
WHERE coffeshop_employeerole.name = 'Bartender' AND coffeshop_employee.is_active = TRUE;
SQL response: Based on the query results, there is no active bartender currently listed.


INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.sql_join_query_engine:> Transformed query given SQL response: None


Transformed query given SQL response: None


In [18]:
from IPython.display import Markdown, display

display(Markdown(response.response))

Based on the query results, there is no active bartender currently listed.